In [27]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import MulticlassMetrics
import decimal
from pyspark import SparkContext,SparkConf
sc=SparkContext()
#Dataset Input
input_data = sc.textFile("./iris.csv")

#Dataset Parse Function
def parse(line):
    field = line.split(",")
    sep_length = float(field[0])
    sep_width = float(field[1])
    pet_length = float(field[2])  
    pet_width = float(field[3])
    tar_class = field[4]
    if tar_class == 'versicolor':
        tarFinal = 0
    else:
        tarFinal = 1
    return LabeledPoint(tarFinal,(sep_length,sep_width,pet_length,pet_width))

#Dataset Parsing
newData = input_data.filter(lambda x : not x.startswith('sepal_length'))
parsed_data = newData.map(parse)

parsed_data.take(4)
  

[LabeledPoint(1.0, [5.1,3.5,1.4,0.2]),
 LabeledPoint(1.0, [4.9,3.0,1.4,0.2]),
 LabeledPoint(1.0, [4.7,3.2,1.3,0.2]),
 LabeledPoint(1.0, [4.6,3.1,1.5,0.2])]

In [ ]:
  
#Dataset Parsing
newData = input_data.filter(lambda x : not x.startswith('sepal_length'))
parsed_data = newData.map(parse)

parsed_data.take(4)

In [20]:
type(parsed_data)

pyspark.rdd.PipelinedRDD

In [ ]:
#TrainTest Split
training, test = parsed_data.randomSplit([0.7, 0.3])

#Model Fit
model = LogisticRegressionWithLBFGS.train(training)

#Model Evaluation
predAndLabel = test.map(lambda x: (float(model.predict(x.features)), x.label))
metrics = MulticlassMetrics(predAndLabel)
true_pos=1.0 * predAndLabel.map(lambda x : (x[0],x[1])).filter(lambda x: x[0] == 1.0).filter(lambda x : x[1] == 1.0).count()
false_pos=1.0 * predAndLabel.map(lambda x : (x[0],x[1])).filter(lambda x: x[0] == 1.0).filter(lambda x : x[1] == 0.0).count()
false_neg = 1.0 * predAndLabel.map(lambda x : (x[0],x[1])).filter(lambda x: x[0] == 0.0).filter(lambda x : x[1] == 1.0).count()
precision = true_pos / (true_pos + false_pos)
recall = true_pos / (true_pos + false_neg)
accuracy = 1.0 * predAndLabel.filter(lambda x: x[0] == x[1]).count() / test.count()
percentage = "%"

#Result
print(f'The accuracy of the model is {100*round(accuracy,2)}{percentage}')
print(f"The model's precision is {100*round(precision,2)}{percentage}")
print(f"The model's recall is {100*round(recall,2)}{percentage}")


In [19]:
sc.stop()

In [28]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

parsed_data = [LabeledPoint(0, [4.6,3.6,1.0,0.2]),  # 3 classes
                LabeledPoint(0, [5.7,4.4,1.5,0.4]),
                LabeledPoint(1, [6.7,3.1,4.4,1.4]),
                LabeledPoint(2, [4.8,3.4,1.6,0.2]),
                LabeledPoint(1, [4.4,3.2,1.3,0.2])]

model = LogisticRegressionWithLBFGS.train(sc.parallelize(parsed_data)) # this will reproduce your error:


Py4JJavaError: An error occurred while calling o527.trainLogisticRegressionModelWithLBFGS.
: org.apache.spark.SparkException: Multinomial models contain a matrix of coefficients, use coefficientMatrix instead.
	at org.apache.spark.ml.classification.LogisticRegressionModel.coefficients(LogisticRegression.scala:929)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.runWithMlLogisticRegression$1(LogisticRegression.scala:455)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.run(LogisticRegression.scala:459)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.run(LogisticRegression.scala:425)
	at org.apache.spark.mllib.classification.LogisticRegressionWithLBFGS.run(LogisticRegression.scala:355)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainRegressionModel(PythonMLLibAPI.scala:92)
	at org.apache.spark.mllib.api.python.PythonMLLibAPI.trainLogisticRegressionModelWithLBFGS(PythonMLLibAPI.scala:308)
	at sun.reflect.NativeMeth

In [29]:

# set numClasses=3:
model = LogisticRegressionWithLBFGS.train(sc.parallelize(parsed_data), numClasses=3) # works OK


In [30]:
type(parsed_data)

list

In [32]:
sc.stop()